# MongoLite Nested Data: (R to MongoDB)
# `Query | Aggregate | Map-Reduce`

**Mr Fugu Data Science**

# (◕‿◕✿)

[youtube](https://www.youtube.com/channel/UCbni-TDI-Ub8VlGaP8HLTNw?view_as=subscriber) | [Github](https://github.com/MrFuguDataScience)


# Outcome & Expectations:

+ Create `Aggregate` function
    + Create Map-Reduce functions
        + Describe use cases for both

# <font color=red>SUBscribe</font> & TURN ON <font color=red>Notification BELL</font>

# <font color=red>LIKE</font> Video 

In [1]:
# library(jsonlite)  # send files to Mongo
library(tidyverse)
library(knitr)     # help run code
library(markdown)  # create markdown files i.e. pdf
library(mongolite) # Create connection/Interface R<-> Mongodb

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  3.0.0     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘readr’ was built under R version 3.4.4”
Warning message:
“package ‘stringr’ was built under R version 3.4.4”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()



# Our Data: 
+ First two candidates:

`{[{'candidate': {'first_name': 'Margaret',
   'last_name': 'Mcdonald',
   'skills': ['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
   'state': 'AL',
   'specialty': 'Machine Learning',
   'experience': 'Mid',
   'relocation': 'no'}},
 {'candidate': {'first_name': 'Michael',
   'last_name': 'Carter',
   'skills': ['TensorFlow', 'R', 'Spark', 'MongoDB', 'C++', 'SQL'],
   'state': 'AR',
   'specialty': 'Machine Learning',
   'experience': 'Junior',
   'relocation': 'yes'}}]`

In [2]:
# Create Connection: 'localhost'

mng_conn<-mongo(collection = 'recruiter_clients',db='berkeley')

#''' Pay attention, you may get an error: `No suitable server`, I got it to work by
# opening the terminal and making a mongod connection
#'''

In [53]:
# Checkout what our data is today: This is NESTED DATA pay attention!

# find all rows:
n<-mng_conn$find('{}')

# get a glimpse (idea)
dplyr::glimpse(n$candidate)

head(n$candidate) # Nested data, outer part is candidate
# head(n$lastModified)

Rows: 500
Columns: 7
$ first_name <chr> "Margaret", "Michael", "Brenda", "Joseph", "Laura", "Chery…
$ last_name  <chr> "Mcdonald", "Carter", "Tyler", "King", "Webb", "Ramirez", …
$ skills     <list> [<"skLearn", "Java", "R", "SQL", "Spark", "C++">, <"Tenso…
$ state      <chr> "AL", "AR", "UT", "FL", "WY", "OK", "NM", "TX", "WI", "UT"…
$ specialty  <chr> "Database", "Data Visualization", "Database", "Statistics"…
$ experience <chr> "Junior", "Junior", "Mid", "Senior", "Junior", "Senior", "…
$ relocation <chr> "no", "yes", "no", "maybe", "maybe", "no", "maybe", "yes",…


,first_name,last_name,skills,state,specialty,experience,relocation
,<chr>,<chr>,<list>,<chr>,<chr>,<chr>,<chr>
1,Margaret,Mcdonald,"skLearn, Java , R , SQL , Spark , C++",AL,Database,Junior,no
2,Michael,Carter,"TensorFlow, R , Spark , MongoDB , C++ , SQL",AR,Data Visualization,Junior,yes
3,Brenda,Tyler,Spark,UT,Database,Mid,no
4,Joseph,King,"skLearn , SQL , R , Spark , Java , C++ , Python , TensorFlow",FL,Statistics,Senior,maybe
5,Laura,Webb,"TensorFlow, C++ , SQL , Java , R , MongoDB",WY,Machine Learning,Junior,maybe
6,Cheryl,Ramirez,"C++ , Python , R , Java , skLearn , SQL , MongoDB , Spark , TensorFlow",OK,Data Visualization,Senior,no


In [54]:
#Weird Output: Hmm?? what do we do?
mng_conn$find(query='{"candidate.first_name":"Brenda"}',
                      fields='{"first_name":1}')

,_id
,<chr>
1,5eb5cc5b47c72ccce7539c1d
2,5eb5cc5b47c72ccce7539d13


In [59]:
mng_conn$count()

brenda<-mng_conn$find('{"candidate.first_name":"Brenda"}',
                      fields='{}')

brenda$candidate
# brenda$`_id`
# What about limiting fields? well let me show you:

[1] 500

,first_name,last_name,skills,state,specialty,experience,relocation
,<chr>,<chr>,<list>,<chr>,<chr>,<chr>,<chr>
1,Brenda,Tyler,Spark,UT,Database,Mid,no
2,Brenda,Kennedy,TensorFlow,NM,Data Visualization,Senior,no


In [6]:
spark<-mng_conn$find(query='{"candidate.skills":["Spark"]}',fields='{}')

spark$candidate

,first_name,last_name,skills,state,specialty,experience,relocation
,<chr>,<chr>,<list>,<chr>,<chr>,<chr>,<chr>
1,Brenda,Tyler,Spark,UT,Database,Mid,no
2,Tiffany,Olson,Spark,MO,Statistics,Mid,maybe
3,Travis,Hawkins,Spark,CO,Statistics,Senior,yes
4,Jack,Chavez,Spark,LA,Data Visualization,Mid,no
5,Christina,Wilson,Spark,CT,Database,Senior,yes
6,Andrew,Fleming,Spark,CT,Machine Learning,Senior,maybe
7,David,Sanchez,Spark,MN,Statistics,Mid,yes
8,Robert,Gray,Spark,NE,Database,Mid,yes
9,Felicia,Mills,Spark,OH,Statistics,Mid,no


In [7]:
C<-mng_conn$find(query='{"candidate.skills":"C++"}',fields='{}')

head(C$candidate)

,first_name,last_name,skills,state,specialty,experience,relocation
,<chr>,<chr>,<list>,<chr>,<chr>,<chr>,<chr>
1,Margaret,Mcdonald,"skLearn, Java , R , SQL , Spark , C++",AL,Database,Junior,no
2,Michael,Carter,"TensorFlow, R , Spark , MongoDB , C++ , SQL",AR,Data Visualization,Junior,yes
3,Joseph,King,"skLearn , SQL , R , Spark , Java , C++ , Python , TensorFlow",FL,Statistics,Senior,maybe
4,Laura,Webb,"TensorFlow, C++ , SQL , Java , R , MongoDB",WY,Machine Learning,Junior,maybe
5,Cheryl,Ramirez,"C++ , Python , R , Java , skLearn , SQL , MongoDB , Spark , TensorFlow",OK,Data Visualization,Senior,no
6,Charles,Stewart,"MongoDB, C++ , Java , SQL , R , Python , skLearn",NM,Machine Learning,Mid,maybe


In [8]:
ididi<-mng_conn$find(query = '{"_id" : {"$oid":"5eb5cc5b47c72ccce7539c1d"}}',fields='{}')

ididi$candidate

,first_name,last_name,skills,state,specialty,experience,relocation
,<chr>,<chr>,<list>,<chr>,<chr>,<chr>,<chr>
1,Brenda,Tyler,Spark,UT,Database,Mid,no


In [9]:
or_<-mng_conn$find(query='{"$or" :[{"candidate.state":"CA"},{"candidate.state":"FL"}]}',
             fields='{}')

head(or_$candidate)


,first_name,last_name,skills,state,specialty,experience,relocation
,<chr>,<chr>,<list>,<chr>,<chr>,<chr>,<chr>
1,Joseph,King,"skLearn , SQL , R , Spark , Java , C++ , Python , TensorFlow",FL,Statistics,Senior,maybe
2,Tammy,Miller,"skLearn , R , TensorFlow, MongoDB , C++ , Java",FL,Data Visualization,Mid,maybe
3,Michael,Carter,"Java, C++",FL,Database,Senior,maybe
4,Sharon,Taylor,"MongoDB, R",CA,Data Visualization,Senior,no
5,Gloria,Rivera,"R , SQL , Java , MongoDB , skLearn , Python , TensorFlow, Spark , C++",CA,Statistics,Senior,maybe
6,Jennifer,Stewart,"Python , Java , skLearn, R , Spark",CA,Statistics,Senior,maybe


In [10]:
all_w_atleast_Python<-mng_conn$find(query='{"candidate.skills": {"$all":["Python"]}}',
                                    fields='{}')
head(all_w_atleast_Python$candidate)

,first_name,last_name,skills,state,specialty,experience,relocation
,<chr>,<chr>,<list>,<chr>,<chr>,<chr>,<chr>
1,Joseph,King,"skLearn , SQL , R , Spark , Java , C++ , Python , TensorFlow",FL,Statistics,Senior,maybe
2,Cheryl,Ramirez,"C++ , Python , R , Java , skLearn , SQL , MongoDB , Spark , TensorFlow",OK,Data Visualization,Senior,no
3,Charles,Stewart,"MongoDB, C++ , Java , SQL , R , Python , skLearn",NM,Machine Learning,Mid,maybe
4,William,Lin,"Python , skLearn, R",WI,Machine Learning,Mid,maybe
5,Robert,Rivera,"Java , skLearn , Python , Spark , TensorFlow, C++ , SQL",PA,Machine Learning,Junior,yes
6,Mr.,James,"TensorFlow, Java , MongoDB , Python , Spark , C++ , R , skLearn , SQL",KY,Data Visualization,Senior,maybe


# Aggregation:


In [33]:
skills_count<-mng_conn$aggregate('[{"$project":{"candidate.first_name":1,
                        "candidate.last_name":1,
                        "numberOfSkills":{"$cond":{"if":{"$isArray":"$candidate.skills"},
                        "then":{"$size":"$candidate.skills"},"else":"NA"}}}}]')


In [34]:
#if you type skills_count$, then press tab: you get 3 options(_id,candidate,numberskills)

head(cbind(skills_count$candidate,numberOfSkillsCount=skills_count$numberOfSkills))
# skills_count$

,first_name,last_name,numberOfSkillsCount
,<chr>,<chr>,<int>
1,Margaret,Mcdonald,6
2,Michael,Carter,6
3,Brenda,Tyler,1
4,Joseph,King,8
5,Laura,Webb,6
6,Cheryl,Ramirez,9


In [60]:
match_ML<-mng_conn$aggregate('[{ "$match":{"candidate.specialty": "Machine Learning"}}]')

head(match_ML$candidate)

,first_name,last_name,skills,state,specialty,experience,relocation
,<chr>,<chr>,<list>,<chr>,<chr>,<chr>,<chr>
1,Laura,Webb,"TensorFlow, C++ , SQL , Java , R , MongoDB",WY,Machine Learning,Junior,maybe
2,Charles,Stewart,"MongoDB, C++ , Java , SQL , R , Python , skLearn",NM,Machine Learning,Mid,maybe
3,William,Lin,"Python , skLearn, R",WI,Machine Learning,Mid,maybe
4,Richard,Woods,"skLearn, MongoDB",UT,Machine Learning,Senior,yes
5,Robert,Rivera,"Java , skLearn , Python , Spark , TensorFlow, C++ , SQL",PA,Machine Learning,Junior,yes
6,Tanya,Schultz,"SQL , MongoDB , R , C++ , Spark , TensorFlow",NV,Machine Learning,Senior,yes


# Map-Reduce:

In [46]:
mpred<-mng_conn$mapreduce(map= 'function(){emit(this.candidate.state,{count:1});}',
reduce = 'function(state,val){
    var value = {count:0};
    for(i=0; i<val.length;i++){
    value.count += val[i].count;}
    return value;}')


In [45]:
head(cbind(states=mpred$`_id`,ppl_per_state=mpred$value))


,states,count
,<fct>,<dbl>
1,AK,10
2,AL,12
3,AR,8
4,AZ,8
5,CA,11
6,CO,10


In [49]:

mng_conn$mapreduce(map = 'function(){ var skill = this.candidate.skills;
for(i in skill){emit(skill[i],1);}}',
reduce = 'function(key,vals){var count=0;
for(i in vals){count += vals[i];} return count;}')

,_id,value
,<chr>,<dbl>
1,C++,274
2,Java,267
3,MongoDB,274
4,Python,273
5,R,272
6,SQL,277
7,Spark,285
8,TensorFlow,280
9,skLearn,267


Mongolite Docs: https://cran.r-project.org/web/packages/mongolite/mongolite.pdf

https://dsotm-rsa.space/post/2019/04/08/mongodb-via-mongolite-r-aggregations-server-tools-and-gridfs/

https://blog.exploratory.io/an-introduction-to-mongodb-query-for-beginners-bd463319aa4c

# ◔̯◔